In [2]:
# Importing packages
import requests
import json
import time

import numpy as np
import pandas as pd

import geopandas as gpd
import fiona
from matplotlib import pyplot as plt
from matplotlib import image as mpimg

In [44]:
api_token = "pk.eyJ1IjoiaXp6eXRhIiwiYSI6ImNsZTZvcjAzcTBvdWQ0MW54dXI2eTE4dDMifQ.ZbpW85x0ou41BuGPuP4F_w"

In [103]:
# creating one data set with cluster boundaries and centers
hilo_centers_kmeans_60 = gpd.read_file('../data/hilo_parcel_cluster_center_60.geojson')
hilo_buffers_kmeans_60 = gpd.read_file('../data/hilo_parcel_cluster_buffer_60.geojson')

hilo_centers_kmeans_125 = gpd.read_file('../data/hilo_parcel_cluster_center_125.geojson')
hilo_buffers_kmeans_125 = gpd.read_file('../data/hilo_parcel_cluster_buffer_125.geojson')

hilo_centers_kmeans_60 = hilo_centers_kmeans_60.rename(columns = {'geometry' : 'center'})
hilo_clusters_kmeans_60 = hilo_centers_kmeans_60.assign(buffer = hilo_buffers_kmeans_60['geometry'])

hilo_clusters_kmeans_60['site_name'] = np.nan
hilo_clusters_kmeans_60['site_location'] = np.nan
hilo_clusters_kmeans_60['site_dist'] = np.nan

hilo_centers_kmeans_125 = hilo_centers_kmeans_125.rename(columns = {'geometry' : 'center'})
hilo_clusters_kmeans_125 = hilo_centers_kmeans_125.assign(buffer = hilo_buffers_kmeans_125['geometry'])

hilo_clusters_kmeans_125['site_name'] = np.nan
hilo_clusters_kmeans_125['site_location'] = np.nan
hilo_clusters_kmeans_125['site_dist'] = np.nan

In [113]:
# creating data sets with cluster boundaries and centers for skater clustering (70 and 140 clusters)
hilo_centers_skater_70 = gpd.read_file('../data/hilo_cluster_centroid_70.geojson')
hilo_buffers_skater_70 = gpd.read_file('../data/hilo_cluster_buffer_skater_70.geojson')

hilo_centers_skater_140 = gpd.read_file('../data/hilo_cluster_centroid_skater_140.geojson')
hilo_buffers_skater_140 = gpd.read_file('../data/hilo_cluster_buffer_skater_140.geojson')

hilo_centers_skater_70 = hilo_centers_skater_70.rename(columns = {'geometry' : 'center'})
hilo_clusters_skater_70 = hilo_centers_skater_70.assign(buffer = hilo_buffers_skater_70['geometry'])

hilo_clusters_skater_70['site_name'] = np.nan
hilo_clusters_skater_70['site_location'] = np.nan
hilo_clusters_skater_70['site_dist'] = np.nan
hilo_clusters_skater_70 = hilo_clusters_skater_70.rename(columns = {'Lat':'lat', 'Long':'long'})

hilo_centers_skater_140 = hilo_centers_skater_140.rename(columns = {'geometry' : 'center'})
hilo_clusters_skater_140 = hilo_centers_skater_140.assign(buffer = hilo_buffers_skater_140['geometry'])

hilo_clusters_skater_140['site_name'] = np.nan
hilo_clusters_skater_140['site_location'] = np.nan
hilo_clusters_skater_140['site_dist'] = np.nan
hilo_clusters_skater_140 = hilo_clusters_skater_140.rename(columns = {'Lat':'lat', 'Long':'long'})

In [46]:
# loading in sites data and selecting for relavent variables (location variables)
hilo_dist = pd.read_csv("../output/complete_hilo.csv")
hilo_col = pd.read_csv("../data/complete_hilo2.csv")


hilo_sites = hilo_dist.append(hilo_col)
hilo_sites = hilo_sites.dropna(subset=['Coordinates'])
hilo_sites = hilo_sites[['Name', 'Address', 'Coordinates']].drop_duplicates()
hilo_sites = hilo_sites.reset_index()

hilo_lat = []
hilo_long = []
for i in hilo_sites["Coordinates"]:
    dic = eval(i)
    try:
        hilo_lat.append(dic["lat"])
    except:
        hilo_lat.append(dic["latitude"])
    try:
        hilo_long.append(dic["lng"])
    except:
        hilo_long.append(dic["longitude"])

hilo_sites = hilo_sites.assign(Lat = hilo_lat, Long = hilo_long)
hilo_sites = gpd.GeoDataFrame(hilo_sites, geometry=gpd.points_from_xy(hilo_sites.Long, hilo_sites.Lat))
hilo_sites = hilo_sites[['Name', 'Address', 'geometry', 'Lat', 'Long']]
hilo_sites = hilo_sites.rename(columns = {'geometry':'Coordinates'})
hilo_sites.head()

/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/2316733467.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hilo_sites = hilo_dist.append(hilo_col)
/Users/izzy/opt/anaconda3/envs/Perpetual/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


,Name,Address,Coordinates,Lat,Long
0,Hilo Bay Medical Clinic,"24 Pukihae Street, Hilo",POINT (-155.08795 19.73014),19.730137,-155.087946
1,Wild Ginger Hotel,"100 Puueo Street, Hilo",POINT (-155.08918 19.72932),19.729316,-155.089175
2,Dolphin Bay Hotel,"333 Iliahi Street, Hilo",POINT (-155.09043 19.72923),19.729229,-155.090427
3,Hilo Bay Oceanfront Bed and Breakfast,"56 Pukihae Street, Hilo",POINT (-155.08816 19.73098),19.730976,-155.088165
4,11 Arruda Ln.,Hilo,POINT (-155.08903 19.72859),19.728593,-155.089030


In [125]:
def find_sites(cluster_data, site_data):
    
    # finding closest dist or collection site for each cluster center within the cluster boundaries

    for idx in range(len(cluster_data)):
        boundary = cluster_data['buffer'][idx]
        #identify sites within the cluster boundaries
        within_boundary = site_data[site_data.Coordinates.within(boundary)].reset_index(drop = True)

        #rest of the code is the same as above basically
        nearby = within_boundary
        distances = []

        #added this if len(nearby) > 0 bc of the troublesome cases where filtering was inaccurate and nothing was nearby
        if len(nearby) > 0:
            
            # when list of nearby businesses is more than 24 long, look at groups of 24 (max size) at a time
            while len(nearby) > 24:
                short_list = nearby.head(24)
                nearby = nearby[24:].reset_index(drop = True)
                
                # creating list of coordiantes for url for api request
                coord = str(cluster_data['long'][idx]) + "," + str(cluster_data['lat'][idx])
                for i in range(len(short_list)):
                    coord = coord + ";" + str(short_list['Long'][i]) +  "," + str(short_list['Lat'][i])
                url = "https://api.mapbox.com/directions-matrix/v1/mapbox/walking/" +\
                        coord + "?sources=0&annotations=distance,duration"
                
                # api request
                r = requests.get(url, params=({'access_token':api_token}))
                r =  json.loads(r.content)
                time.sleep(1)
                
                # isolating walking distances from api request data
                for d in r['durations'][0][1:]:
                    distances.append(d)
            
            # when list of businesses is less than 24, look through remaining businesses (same process as above)
            coord = str(cluster_data['long'][idx]) + "," + str(cluster_data['lat'][idx])
            for i in range(len(nearby)):
                coord = coord + ";" + str(nearby['Long'][i]) +  "," + str(nearby['Lat'][i])
            url = "https://api.mapbox.com/directions-matrix/v1/mapbox/walking/" +\
                    coord + "?sources=0&annotations=distance,duration"

            r = requests.get(url, params=({'access_token':api_token}))
            r =  json.loads(r.content)

            for d in r['durations'][0][1:]:
                distances.append(d)
            
            # append distances to business data, and select the closest one to the centroid center
            within_boundary = within_boundary.assign(Dist = distances)
            closest = within_boundary[within_boundary.Dist == within_boundary.Dist.min()]
            
            #update cluster_data with the site name, location, and walking distance
            cluster_data['site_name'][idx] = closest['Name'].values[0]
            cluster_data['site_location'][idx] = closest['Coordinates'].values[0]
            cluster_data['site_dist'][idx] = closest['Dist'].values[0]
        
        #if no nearby businesses, fill in sitr name, location, and walking distance with nan
        else:
            cluster_data['site_name'][idx] = np.nan
            cluster_data['site_location'][idx] = np.nan
            cluster_data['site_dist'][idx] = np.nan
    
    # creating seperates data frames for clusters where a suitable site was
    # found and for clusters where no site was found or the site was quite far
    clusters_nan = cluster_data[cluster_data['site_name'].isnull()]
    clusters_far = cluster_data[cluster_data['site_dist'] > 300]
    clusters_redo = pd.concat([clusters_far, clusters_nan])
    clusters_redo = clusters_redo.reset_index()

    clusters_found = cluster_data[cluster_data['site_name'].notnull()]
    clusters_found = clusters_found[clusters_found['site_dist'] <= 300]
    
    # finding closest dist or collection site for each cluster center with no or far sites

    for idx in range(len(clusters_redo)):  
        nearby = site_data 

        lat = clusters_redo.lat[idx]
        long = clusters_redo.long[idx]
        nearby['euc_diff'] = ((nearby['Lat']-lat)**2 + (nearby['Long']-long)**2)**0.5
        nearby = nearby.nsmallest(20, ['euc_diff'])
        nearby = nearby.reset_index()

        #create a list to store distances
        distances = []
        
        # create url w coorindates for api request
        coord = str(clusters_redo['long'][idx]) + "," + str(clusters_redo['lat'][idx])
        for i in range(len(nearby)):
            coord = coord + ";" + str(nearby['Long'][i]) +  "," + str(nearby['Lat'][i])
        url = "https://api.mapbox.com/directions-matrix/v1/mapbox/walking/" +\
                coord + "?sources=0&annotations=distance,duration"
    
        # api request
        r = requests.get(url, params=({'access_token':api_token}))
        r =  json.loads(r.content)
        time.sleep(1)

        # add distances to list of distances
        for d in r['durations'][0][1:]:
            distances.append(d)

        # add distances as a column to our copied version of site_data data
        nearby = nearby.assign(Dist = distances)
        # extract the row with the closest distance
        closest = nearby[nearby.Dist == nearby.Dist.min()]

        # add info from that closest row to our hilo_clusters data
        clusters_redo['site_name'][idx] = closest['Name'].values[0]
        clusters_redo['site_location'][idx] = closest['Coordinates'].values[0]
        clusters_redo['site_dist'][idx] = closest['Dist'].values[0]

    # put all custers back together in a data frame now that they all have an assigned site
    combined_clusters = pd.concat([clusters_found, clusters_redo])
    combined_clusters = combined_clusters.reset_index()
    combined_clusters = combined_clusters[['lat', 'long', 'center', 'buffer', 'site_name', 'site_location', 'site_dist']]
    
    return combined_clusters
    #hilo_clusters.to_csv('hilo_cluster_collection_sites.csv')

In [78]:
hilo_clusters_kmeans_60 = find_sites(hilo_clusters_kmeans_60, hilo_sites)
hilo_clusters_kmeans_60.to_csv('../output/hilo_cluster_collection_sites_kmeans_60.csv')

/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1715303602.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_name'][idx] = closest['Name'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1715303602.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_location'][idx] = closest['Coordinates'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1715303602.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

    level_0        lat        long                       center  \
0         0  19.708217 -155.083525  POINT (-155.08353 19.70822)   
1         1  19.677140 -155.108048  POINT (-155.10805 19.67714)   
2         2  19.732046 -155.026875  POINT (-155.02687 19.73205)   
3         3  19.709272 -155.074447  POINT (-155.07445 19.70927)   
4         4  19.726451 -155.062554  POINT (-155.06255 19.72645)   
5         5  19.724957 -155.090191  POINT (-155.09019 19.72496)   
6         6  19.699883 -155.088063  POINT (-155.08806 19.69988)   
7         7  19.731083 -155.042241  POINT (-155.04224 19.73108)   
8         8  19.679786 -155.096999  POINT (-155.09700 19.67979)   
9         9  19.672582 -155.074406  POINT (-155.07441 19.67258)   
10       10  19.709709 -155.127372  POINT (-155.12737 19.70971)   
11       11  19.682687 -155.183060  POINT (-155.18306 19.68269)   
12       12  19.715909 -155.079790  POINT (-155.07979 19.71591)   
13       13  19.699032 -155.080119  POINT (-155.08012 19.69903

/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1715303602.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clusters_redo['site_name'][idx] = closest['Name'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1715303602.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clusters_redo['site_location'][idx] = closest['Coordinates'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1715303602.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

In [109]:
# creating geojson file for collection sites
hilo_sites_kmeans_60 = hilo_clusters_kmeans_60.iloc[:,[4,5,6]]
hilo_sites_kmeans_60 = gpd.GeoDataFrame(hilo_sites_kmeans_60, geometry=hilo_sites_kmeans_60.site_location)
hilo_sites_kmeans_60 = hilo_sites_kmeans_60.drop(['site_location'],axis=1)
hilo_sites_kmeans_60.to_file("hilo_cluster_collection_sites_60.geojson",driver="GeoJSON")

# creating geojson file for buffers around collection sites
buffer = hilo_sites_kmeans_60.buffer(0.002)
hilo_copy = hilo_sites_kmeans_60[:]
hilo_copy["geometry"] = buffer
hilo_copy.to_file("hilo_cluster_collection_sites_buffer_60.geojson",driver="GeoJSON")

/Users/izzy/opt/anaconda3/envs/Perpetual/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/izzy/opt/anaconda3/envs/Perpetual/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [105]:
hilo_clusters_kmeans_125= find_sites(hilo_clusters_kmeans_125, hilo_sites)
hilo_clusters_kmeans_125.to_csv('../output/hilo_cluster_collection_sites_kmeans_125.csv')

In [107]:
# creating geojson file for collection sites
hilo_sites_kmeans_125 = hilo_clusters_kmeans_125.iloc[:,[4,5,6]]
hilo_sites_kmeans_125 = gpd.GeoDataFrame(hilo_sites_kmeans_125, geometry=hilo_sites_kmeans_125.site_location)
hilo_sites_kmeans_125 = hilo_sites_kmeans_125.drop(['site_location'],axis=1)
hilo_sites_kmeans_125.to_file("../output/hilo_cluster_collection_sites_125.geojson",driver="GeoJSON")

# creating geojson file for buffers around collection sites
buffer = hilo_sites_kmeans_125.buffer(0.002)
hilo_copy = hilo_sites_kmeans_125[:]
hilo_copy["geometry"] = buffer
hilo_copy.to_file("../output/hilo_cluster_collection_sites_buffer_125.geojson",driver="GeoJSON")

/Users/izzy/opt/anaconda3/envs/Perpetual/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/izzy/opt/anaconda3/envs/Perpetual/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [116]:
hilo_clusters_skater_70 = find_sites(hilo_clusters_skater_70, hilo_sites)
hilo_clusters_skater_70.to_csv('../output/hilo_cluster_collection_sites_skater_70.csv')

/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1321602970.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_name'][idx] = closest['Name'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1321602970.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_location'][idx] = closest['Coordinates'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1321602970.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [128]:
# creating geojson file for collection sites
hilo_sites_skater_70 = hilo_clusters_skater_70.iloc[:,[4,5,6]]
hilo_sites_skater_70 = gpd.GeoDataFrame(hilo_sites_skater_70, geometry=hilo_sites_skater_70.site_location)
hilo_sites_skater_70 = hilo_sites_skater_70.drop(['site_location'],axis=1)
hilo_sites_skater_70.to_file("../output/hilo_cluster_collection_sites_skater_70.geojson",driver="GeoJSON")

# creating geojson file for buffers around collection sites
buffer = hilo_sites_skater_70.buffer(0.002)
hilo_copy = hilo_sites_skater_70[:]
hilo_copy["geometry"] = buffer
hilo_copy.to_file("../output/hilo_cluster_collection_sites_buffer_skater_70.geojson",driver="GeoJSON")

/Users/izzy/opt/anaconda3/envs/Perpetual/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/izzy/opt/anaconda3/envs/Perpetual/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [126]:
hilo_clusters_skater_140 = find_sites(hilo_clusters_skater_140, hilo_sites)
hilo_clusters_skater_140.to_csv('../output/hilo_cluster_collection_sites_skater_140.csv')

0
1
2
3
4
5
6
7
8


/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_name'][idx] = np.nan
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_location'][idx] = np.nan
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

9
10
11


/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_name'][idx] = closest['Name'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_location'][idx] = closest['Coordinates'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

12
13
14


...

127
128
129
130
131
132
133
134
135
136
137
138
139


/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_name'][idx] = closest['Name'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['site_location'][idx] = closest['Coordinates'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

140
141
142
143
144
145


/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clusters_redo['site_name'][idx] = closest['Name'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clusters_redo['site_location'][idx] = closest['Coordinates'].values[0]
/var/folders/wf/2tp90k6928s17ms0shcykz840000gn/T/ipykernel_5679/1783455687.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

OSError: Cannot save file into a non-existent directory: '../output'

In [129]:
# creating geojson file for collection sites
hilo_sites_skater_140 = hilo_clusters_skater_140.iloc[:,[4,5,6]]
hilo_sites_skater_140 = gpd.GeoDataFrame(hilo_sites_skater_140, geometry=hilo_sites_skater_140.site_location)
hilo_sites_skater_140 = hilo_sites_skater_140.drop(['site_location'],axis=1)
hilo_sites_skater_140.to_file("../output/hilo_cluster_collection_sites_skater_140.geojson",driver="GeoJSON")

# creating geojson file for buffers around collection sites
buffer = hilo_sites_skater_140.buffer(0.002)
hilo_copy = hilo_sites_skater_140[:]
hilo_copy["geometry"] = buffer
hilo_copy.to_file("../output/hilo_cluster_collection_sites_buffer_skater_140.geojson",driver="GeoJSON")

/Users/izzy/opt/anaconda3/envs/Perpetual/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/izzy/opt/anaconda3/envs/Perpetual/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
